In [12]:
import requests
import sqlite3
import os
from dotenv import load_dotenv
import sqlite3
import time

# Load environment variables
load_dotenv()

# Get API keys from environment variable and split into a list
api_keys = os.getenv('API_KEY_ALPHAVANTAGE_sbabel_umass_edu', '').split(',')
if not api_keys or api_keys[0] == '':
    raise ValueError("No API keys found in the environment variable 'API_KEYS'.")


# 1. Read tickers from a .txt file

with open('../data/top60tickers.txt', 'r') as f:
    content = f.read().strip()
    # Remove any surrounding spaces and then split by comma
    tickers = [ticker.strip().strip("'").strip('"') for ticker in content.split(',') if ticker.strip()]


# Function to get the current API key based on index
current_key_index = 0
def get_current_api_key():
    return api_keys[current_key_index].strip()

# Function to rotate to the next API key
def rotate_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(api_keys)
    print(f"Rotated to new API key: {get_current_api_key()}")


In [29]:
# 2. Connect to SQLite database
conn = sqlite3.connect('../data/dbs/company_overview.db')
cursor = conn.cursor()

In [30]:
# 3. Create table to store company overview data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS company_overview (
        Symbol TEXT PRIMARY KEY,
        AssetType TEXT,
        Name TEXT,
        Description TEXT,
        CIK TEXT,
        Exchange TEXT,
        Currency TEXT,
        Country TEXT,
        Sector TEXT,
        Industry TEXT,
        Address TEXT,
        OfficialSite TEXT,
        FiscalYearEnd TEXT,
        LatestQuarter TEXT,
        MarketCapitalization TEXT,
        EBITDA TEXT,
        PERatio TEXT,
        PEGRatio TEXT,
        BookValue TEXT,
        DividendPerShare TEXT,
        DividendYield TEXT,
        EPS TEXT,
        RevenuePerShareTTM TEXT,
        ProfitMargin TEXT,
        OperatingMarginTTM TEXT,
        ReturnOnAssetsTTM TEXT,
        ReturnOnEquityTTM TEXT,
        RevenueTTM TEXT,
        GrossProfitTTM TEXT,
        DilutedEPSTTM TEXT,
        QuarterlyEarningsGrowthYOY TEXT,
        QuarterlyRevenueGrowthYOY TEXT,
        AnalystTargetPrice TEXT,
        AnalystRatingStrongBuy TEXT,
        AnalystRatingBuy TEXT,
        AnalystRatingHold TEXT,
        AnalystRatingSell TEXT,
        AnalystRatingStrongSell TEXT,
        TrailingPE TEXT,
        ForwardPE TEXT,
        PriceToSalesRatioTTM TEXT,
        PriceToBookRatio TEXT,
        EVToRevenue TEXT,
        EVToEBITDA TEXT,
        Beta TEXT,
        Week52High TEXT,
        Week52Low TEXT,
        MovingAverage50Day TEXT,
        MovingAverage200Day TEXT,
        SharesOutstanding TEXT,
        DividendDate TEXT,
        ExDividendDate TEXT
    )
''')
conn.commit()

In [4]:
overview_arr = []

In [25]:

# Loop through each ticker and fetch data using the current API key
for ticker in tickers:
    print(f"Fetching data for {ticker} using API key {get_current_api_key()}...")
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
    
    # Try to fetch data; if a limit error occurs, rotate to the next API key and retry
    success = False
    attempts = 0
    max_attempts = len(api_keys)  # Try each key once at most
    while not success and attempts < max_attempts:
        r = requests.get(url)
        data = r.json()
        
        # Check for expected key or for a note/error message indicating a rate limit
        if data and 'Symbol' in data:
            success = True
        elif data.get('Note') or data.get('Error Message'):
            print(f"API limit reached or error encountered for ticker {ticker} with key {get_current_api_key()}.")
            rotate_api_key()  # Switch API key
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
        else:
            print(f"No valid data for {ticker} using key {get_current_api_key()}.")
            rotate_api_key()
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
    
    if not success:
        print(f"Skipping {ticker} after trying all API keys.")
        continue

    # Prepare the overview data (defaulting missing fields to empty strings)
    overview = {
        'Symbol': data.get("Symbol", ""),
        'AssetType': data.get("AssetType", ""),
        'Name': data.get("Name", ""),
        'Description': data.get("Description", ""),
        'CIK': data.get("CIK", ""),
        'Exchange': data.get("Exchange", ""),
        'Currency': data.get("Currency", ""),
        'Country': data.get("Country", ""),
        'Sector': data.get("Sector", ""),
        'Industry': data.get("Industry", ""),
        'Address': data.get("Address", ""),
        'OfficialSite': data.get("OfficialSite", ""),
        'FiscalYearEnd': data.get("FiscalYearEnd", ""),
        'LatestQuarter': data.get("LatestQuarter", ""),
        'MarketCapitalization': data.get("MarketCapitalization", ""),
        'EBITDA': data.get("EBITDA", ""),
        'PERatio': data.get("PERatio", ""),
        'PEGRatio': data.get("PEGRatio", ""),
        'BookValue': data.get("BookValue", ""),
        'DividendPerShare': data.get("DividendPerShare", ""),
        'DividendYield': data.get("DividendYield", ""),
        'EPS': data.get("EPS", ""),
        'RevenuePerShareTTM': data.get("RevenuePerShareTTM", ""),
        'ProfitMargin': data.get("ProfitMargin", ""),
        'OperatingMarginTTM': data.get("OperatingMarginTTM", ""),
        'ReturnOnAssetsTTM': data.get("ReturnOnAssetsTTM", ""),
        'ReturnOnEquityTTM': data.get("ReturnOnEquityTTM", ""),
        'RevenueTTM': data.get("RevenueTTM", ""),
        'GrossProfitTTM': data.get("GrossProfitTTM", ""),
        'DilutedEPSTTM': data.get("DilutedEPSTTM", ""),
        'QuarterlyEarningsGrowthYOY': data.get("QuarterlyEarningsGrowthYOY", ""),
        'QuarterlyRevenueGrowthYOY': data.get("QuarterlyRevenueGrowthYOY", ""),
        'AnalystTargetPrice': data.get("AnalystTargetPrice", ""),
        'AnalystRatingStrongBuy': data.get("AnalystRatingStrongBuy", ""),
        'AnalystRatingBuy': data.get("AnalystRatingBuy", ""),
        'AnalystRatingHold': data.get("AnalystRatingHold", ""),
        'AnalystRatingSell': data.get("AnalystRatingSell", ""),
        'AnalystRatingStrongSell': data.get("AnalystRatingStrongSell", ""),
        'TrailingPE': data.get("TrailingPE", ""),
        'ForwardPE': data.get("ForwardPE", ""),
        'PriceToSalesRatioTTM': data.get("PriceToSalesRatioTTM", ""),
        'PriceToBookRatio': data.get("PriceToBookRatio", ""),
        'EVToRevenue': data.get("EVToRevenue", ""),
        'EVToEBITDA': data.get("EVToEBITDA", ""),
        'Beta': data.get("Beta", ""),
        'Week52High': data.get("52WeekHigh", ""),
        'Week52Low': data.get("52WeekLow", ""),
        'MovingAverage50Day': data.get("50DayMovingAverage", ""),
        'MovingAverage200Day': data.get("200DayMovingAverage", ""),
        'SharesOutstanding': data.get("SharesOutstanding", ""),
        'DividendDate': data.get("DividendDate", ""),
        'ExDividendDate': data.get("ExDividendDate", "")
    }

    overview_arr.append(overview)


Fetching data for AAPL using API key KP571VIBG347AJAA...
Fetching data for MSFT using API key KP571VIBG347AJAA...
Fetching data for NVDA using API key KP571VIBG347AJAA...
Fetching data for AMZN using API key KP571VIBG347AJAA...
Fetching data for GOOG using API key KP571VIBG347AJAA...
Fetching data for GOOGL using API key KP571VIBG347AJAA...
Fetching data for META using API key KP571VIBG347AJAA...
Fetching data for BRK-A using API key KP571VIBG347AJAA...
Fetching data for BRK-B using API key KP571VIBG347AJAA...
Fetching data for TSM using API key KP571VIBG347AJAA...
Fetching data for AVGO using API key KP571VIBG347AJAA...
Fetching data for TSLA using API key KP571VIBG347AJAA...
Fetching data for LLY using API key KP571VIBG347AJAA...
Fetching data for WMT using API key KP571VIBG347AJAA...
Fetching data for JPM using API key KP571VIBG347AJAA...
Fetching data for V using API key KP571VIBG347AJAA...
Fetching data for XOM using API key KP571VIBG347AJAA...
Fetching data for MA using API key K

KeyboardInterrupt: 

In [32]:
len(overview_arr)

25

In [ ]:
import requests
import sqlite3
import os
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variable and split into a list
api_keys = os.getenv('API_KEY_ALPHAVANTAGE_sbabel_umass_edu', '').split(',')
if not api_keys or api_keys[0] == '':
    raise ValueError("No API keys found in the environment variable 'API_KEYS'.")

# Function to get the current API key based on index
current_key_index = 0
def get_current_api_key():
    return api_keys[current_key_index].strip()

# Function to rotate to the next API key
def rotate_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(api_keys)
    print(f"Rotated to new API key: {get_current_api_key()}")

# Read tickers from the text file (a single line with comma-separated tickers)
with open('../data/top60tickers.txt', 'r') as f:
    content = f.read().strip()
    tickers = [ticker.strip().strip("'").strip('"') for ticker in content.split(',') if ticker.strip()]

# Connect to SQLite database and create a table for company overviews
conn = sqlite3.connect('company_overview.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS company_overview (
        Symbol TEXT PRIMARY KEY,
        AssetType TEXT,
        Name TEXT,
        Description TEXT,
        CIK TEXT,
        Exchange TEXT,
        Currency TEXT,
        Country TEXT,
        Sector TEXT,
        Industry TEXT,
        Address TEXT,
        OfficialSite TEXT,
        FiscalYearEnd TEXT,
        LatestQuarter TEXT,
        MarketCapitalization TEXT,
        EBITDA TEXT,
        PERatio TEXT,
        PEGRatio TEXT,
        BookValue TEXT,
        DividendPerShare TEXT,
        DividendYield TEXT,
        EPS TEXT,
        RevenuePerShareTTM TEXT,
        ProfitMargin TEXT,
        OperatingMarginTTM TEXT,
        ReturnOnAssetsTTM TEXT,
        ReturnOnEquityTTM TEXT,
        RevenueTTM TEXT,
        GrossProfitTTM TEXT,
        DilutedEPSTTM TEXT,
        QuarterlyEarningsGrowthYOY TEXT,
        QuarterlyRevenueGrowthYOY TEXT,
        AnalystTargetPrice TEXT,
        AnalystRatingStrongBuy TEXT,
        AnalystRatingBuy TEXT,
        AnalystRatingHold TEXT,
        AnalystRatingSell TEXT,
        AnalystRatingStrongSell TEXT,
        TrailingPE TEXT,
        ForwardPE TEXT,overview_arr
        PriceToSalesRatioTTM TEXT,
        PriceToBookRatio TEXT,
        EVToRevenue TEXT,
        EVToEBITDA TEXT,
        Beta TEXT,
        Week52High TEXT,
        Week52Low TEXT,
        MovingAverage50Day TEXT,
        MovingAverage200Day TEXT,
        SharesOutstanding TEXT,
        DividendDate TEXT,
        ExDividendDate TEXT
    )
''')
conn.commit()

# Loop through each ticker and fetch data using the current API key
for ticker in tickers:
    print(f"Fetching data for {ticker} using API key {get_current_api_key()}...")
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
    
    # Try to fetch data; if a limit error occurs, rotate to the next API key and retry
    success = False
    attempts = 0
    max_attempts = len(api_keys)  # Try each key once at most
    while not success and attempts < max_attempts:
        r = requests.get(url)
        data = r.json()
        
        # Check for expected key or for a note/error message indicating a rate limit
        if data and 'Symbol' in data:
            success = True
        elif data.get('Note') or data.get('Error Message'):
            print(f"API limit reached or error encountered for ticker {ticker} with key {get_current_api_key()}.")
            rotate_api_key()  # Switch API key
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
            # Optionally wait a bit before retrying (e.g., to respect rate limits)
            time.sleep(12)
        else:
            print(f"No valid data for {ticker} using key {get_current_api_key()}.")
            rotate_api_key()
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
            time.sleep(12)
    
    if not success:
        print(f"Skipping {ticker} after trying all API keys.")
        continue

    # Prepare the overview data (defaulting missing fields to empty strings)
    overview = {
        'Symbol': data.get("Symbol", ""),
        'AssetType': data.get("AssetType", ""),
        'Name': data.get("Name", ""),
        'Description': data.get("Description", ""),
        'CIK': data.get("CIK", ""),
        'Exchange': data.get("Exchange", ""),
        'Currency': data.get("Currency", ""),
        'Country': data.get("Country", ""),
        'Sector': data.get("Sector", ""),
        'Industry': data.get("Industry", ""),
        'Address': data.get("Address", ""),
        'OfficialSite': data.get("OfficialSite", ""),
        'FiscalYearEnd': data.get("FiscalYearEnd", ""),
        'LatestQuarter': data.get("LatestQuarter", ""),
        'MarketCapitalization': data.get("MarketCapitalization", ""),
        'EBITDA': data.get("EBITDA", ""),
        'PERatio': data.get("PERatio", ""),
        'PEGRatio': data.get("PEGRatio", ""),
        'BookValue': data.get("BookValue", ""),
        'DividendPerShare': data.get("DividendPerShare", ""),
        'DividendYield': data.get("DividendYield", ""),
        'EPS': data.get("EPS", ""),
        'RevenuePerShareTTM': data.get("RevenuePerShareTTM", ""),
        'ProfitMargin': data.get("ProfitMargin", ""),
        'OperatingMarginTTM': data.get("OperatingMarginTTM", ""),
        'ReturnOnAssetsTTM': data.get("ReturnOnAssetsTTM", ""),
        'ReturnOnEquityTTM': data.get("ReturnOnEquityTTM", ""),
        'RevenueTTM': data.get("RevenueTTM", ""),
        'GrossProfitTTM': data.get("GrossProfitTTM", ""),
        'DilutedEPSTTM': data.get("DilutedEPSTTM", ""),
        'QuarterlyEarningsGrowthYOY': data.get("QuarterlyEarningsGrowthYOY", ""),
        'QuarterlyRevenueGrowthYOY': data.get("QuarterlyRevenueGrowthYOY", ""),
        'AnalystTargetPrice': data.get("AnalystTargetPrice", ""),
        'AnalystRatingStrongBuy': data.get("AnalystRatingStrongBuy", ""),
        'AnalystRatingBuy': data.get("AnalystRatingBuy", ""),
        'AnalystRatingHold': data.get("AnalystRatingHold", ""),
        'AnalystRatingSell': data.get("AnalystRatingSell", ""),
        'AnalystRatingStrongSell': data.get("AnalystRatingStrongSell", ""),
        'TrailingPE': data.get("TrailingPE", ""),
        'ForwardPE': data.get("ForwardPE", ""),
        'PriceToSalesRatioTTM': data.get("PriceToSalesRatioTTM", ""),
        'PriceToBookRatio': data.get("PriceToBookRatio", ""),
        'EVToRevenue': data.get("EVToRevenue", ""),
        'EVToEBITDA': data.get("EVToEBITDA", ""),
        'Beta': data.get("Beta", ""),
        'Week52High': data.get("52WeekHigh", ""),
        'Week52Low': data.get("52WeekLow", ""),
        'MovingAverage50Day': data.get("50DayMovingAverage", ""),
        'MovingAverage200Day': data.get("200DayMovingAverage", ""),
        'SharesOutstanding': data.get("SharesOutstanding", ""),
        'DividendDate': data.get("DividendDate", ""),
        'ExDividendDate': data.get("ExDividendDate", "")
    }
    overview_arr.append(overview)
    # # Insert the data into the SQLite database
    # cursor.execute('''
    #     INSERT OR REPLACE INTO company_overview (
    #         Symbol, AssetType, Name, Description, CIK, Exchange, Currency, Country, Sector,
    #         Industry, Address, OfficialSite, FiscalYearEnd, LatestQuarter, MarketCapitalization,
    #         EBITDA, PERatio, PEGRatio, BookValue, DividendPerShare, DividendYield, EPS,
    #         RevenuePerShareTTM, ProfitMargin, OperatingMarginTTM, ReturnOnAssetsTTM, ReturnOnEquityTTM,
    #         RevenueTTM, GrossProfitTTM, DilutedEPSTTM, QuarterlyEarningsGrowthYOY, QuarterlyRevenueGrowthYOY,
    #         AnalystTargetPrice, AnalystRatingStrongBuy, AnalystRatingBuy, AnalystRatingHold,
    #         AnalystRatingSell, AnalystRatingStrongSell, TrailingPE, ForwardPE, PriceToSalesRatioTTM,import subprocess

def connect_nordvpn(country='United_States'):
    # Connect to a server in a specific country
    result = subprocess.run(["nordvpn", "connect", country], capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("Error connecting:", result.stderr)

def disconnect_nordvpn():
    result = subprocess.run(["nordvpn", "disconnect"], capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("Error disconnecting:", result.stderr)

# # Example usage
# connect_nordvpn('United_States')
# # Do your proxy requests here...
# disconnect_nordvpn()

    #         PriceToBookRatio, EVToRevenue, EVToEBITDA, Beta, Week52High, Week52Low,
    #         MovingAverage50Day, MovingAverage200Day, SharesOutstanding, DividendDate, ExDividendDate
    #     ) VALUES (
    #         :Symbol, :AssetType, :Name, :Description, :CIK, :Exchange, :Currency, :Country, :Sector,
    #         :Industry, :Address, :OfficialSite, :FiscalYearEnd, :LatestQuarter, :MarketCapitalization,
    #         :EBITDA, :PERatio, :PEGRatio, :BookValue, :DividendPerShare, :DividendYield, :EPS,
    #         :RevenuePerShareTTM, :ProfitMargin, :OperatingMarginTTM, :ReturnOnAssetsTTM, :ReturnOnEquityTTM,
    #         :RevenueTTM, :GrossProfitTTM, :DilutedEPSTTM, :QuarterlyEarningsGrowthYOY, :QuarterlyRevenueGrowthYOY,
    #         :AnalystTargetPrice, :AnalystRatingStrongBuy, :AnalystRatingBuy, :AnalystRatingHold,
    #         :AnalystRatingSell, :AnalystRatingStrongSell, :TrailingPE, :ForwardPE, :PriceToSalesRatioTTM,
    #         :PriceToBookRatio, :EVToRevenue, :EVToEBITDA, :Beta, :Week52High, :Week52Low,
    #         :MovingAverage50Day, :MovingAverage200Day, :SharesOutstanding, :DividendDate, :ExDividendDate
    #     )
    # ''', overview)
    
    # conn.commit()
    # Optionally wait a bit between ticker requests to avoid rapid-fire requests
    time.sleep(12)

# Close the database connection
conn.close()
print("Data for all tickers has been saved to the database.")


Fetching data for AAPL using API key QZEDRTVXR9MJO8IW...
Fetching data for MSFT using API key QZEDRTVXR9MJO8IW...
Fetching data for NVDA using API key QZEDRTVXR9MJO8IW...
Fetching data for AMZN using API key QZEDRTVXR9MJO8IW...
Fetching data for GOOG using API key QZEDRTVXR9MJO8IW...
Fetching data for GOOGL using API key QZEDRTVXR9MJO8IW...
Fetching data for META using API key QZEDRTVXR9MJO8IW...
Fetching data for BRK-A using API key QZEDRTVXR9MJO8IW...
Fetching data for BRK-B using API key QZEDRTVXR9MJO8IW...
Fetching data for TSM using API key QZEDRTVXR9MJO8IW...
Fetching data for AVGO using API key QZEDRTVXR9MJO8IW...
Fetching data for TSLA using API key QZEDRTVXR9MJO8IW...
Fetching data for LLY using API key QZEDRTVXR9MJO8IW...
Fetching data for WMT using API key QZEDRTVXR9MJO8IW...
Fetching data for JPM using API key QZEDRTVXR9MJO8IW...
Fetching data for V using API key QZEDRTVXR9MJO8IW...
Fetching data for XOM using API key QZEDRTVXR9MJO8IW...
Fetching data for MA using API key Q

KeyboardInterrupt: 

In [ ]:

# Insert the data into the SQLite database
cursor.execute('''
    INSERT OR REPLACE INTO company_overview (
        Symbol, AssetType, Name, Description, CIK, Exchange, Currency, Country, Sector,
        Industry, Address, OfficialSite, FiscalYearEnd, LatestQuarter, MarketCapitalization,
        EBITDA, PERatio, PEGRatio, BookValue, DividendPerShare, DividendYield, EPS,
        RevenuePerShareTTM, ProfitMargin, OperatingMarginTTM, ReturnOnAssetsTTM, ReturnOnEquityTTM,
        RevenueTTM, GrossProfitTTM, DilutedEPSTTM, QuarterlyEarningsGrowthYOY, QuarterlyRevenueGrowthYOY,
        AnalystTargetPrice, AnalystRatingStrongBuy, AnalystRatingBuy, AnalystRatingHold,
        AnalystRatingSell, AnalystRatingStrongSell, TrailingPE, ForwardPE, PriceToSalesRatioTTM,
        PriceToBookRatio, EVToRevenue, EVToEBITDA, Beta, Week52High, Week52Low,
        MovingAverage50Day, MovingAverage200Day, SharesOutstanding, DividendDate, ExDividendDate
    ) VALUES (
        :Symbol, :AssetType, :Name, :Description, :CIK, :Exchange, :Currency, :Country, :Sector,
        :Industry, :Address, :OfficialSite, :FiscalYearEnd, :LatestQuarter, :MarketCapitalization,
        :EBITDA, :PERatio, :PEGRatio, :BookValue, :DividendPerShare, :DividendYield, :EPS,
        :RevenuePerShareTTM, :ProfitMargin, :OperatingMarginTTM, :ReturnOnAssetsTTM, :ReturnOnEquityTTM,
        :RevenueTTM, :GrossProfitTTM, :DilutedEPSTTM, :QuarterlyEarningsGrowthYOY, :QuarterlyRevenueGrowthYOY,
        :AnalystTargetPrice, :AnalystRatingStrongBuy, :AnalystRatingBuy, :AnalystRatingHold,
        :AnalystRatingSell, :AnalystRatingStrongSell, :TrailingPE, :ForwardPE, :PriceToSalesRatioTTM,
        :PriceToBookRatio, :EVToRevenue, :EVToEBITDA, :Beta, :Week52High, :Week52Low,
        :MovingAverage50Day, :MovingAverage200Day, :SharesOutstanding, :DividendDate, :ExDividendDate
    )
''', overview)

conn.commit()

# Close the database connection
conn.close()
print("Data for all tickers has been saved to the database.")


In [10]:
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol=AAPL&apikey=SUEC1VC9Y2Y2XCE7'
    
r = requests.get(url)
data = r.json()

NameError: name 'requests' is not defined

In [11]:
data

NameError: name 'data' is not defined

In [13]:
from fetch_funcs import fetch_company_overview
from helpers import connect_nordvpn, disconnect_nordvpn
import requests
import sqlite3
import os
import time
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variable and split into a list
api_keys = os.getenv('API_KEY_ALPHAVANTAGE_sbabel_umass_edu', '').split(',')
if not api_keys or api_keys[0] == '':
    raise ValueError("No API keys found in the environment variable 'API_KEYS'.")

# Function to get the current API key based on index
current_key_index = 0
def get_current_api_key():
    return api_keys[current_key_index].strip()

# Function to rotate to the next API key
def rotate_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(api_keys)
    print(f"Rotated to new API key: {get_current_api_key()}")

# Read tickers from the text file (a single line with comma-separated tickers)
with open('../data/top60tickers.txt', 'r') as f:
    content = f.read().strip()
    tickers = [ticker.strip().strip("'").strip('"') for ticker in content.split(',') if ticker.strip()]

# Connect to SQLite database and create a table for company overviews
conn = sqlite3.connect('company_overview.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS company_overview (
        Symbol TEXT PRIMARY KEY,
        AssetType TEXT,
        Name TEXT,
        Description TEXT,
        CIK TEXT,
        Exchange TEXT,
        Currency TEXT,
        Country TEXT,
        Sector TEXT,
        Industry TEXT,
        Address TEXT,
        OfficialSite TEXT,
        FiscalYearEnd TEXT,
        LatestQuarter TEXT,
        MarketCapitalization TEXT,
        EBITDA TEXT,
        PERatio TEXT,
        PEGRatio TEXT,
        BookValue TEXT,
        DividendPerShare TEXT,
        DividendYield TEXT,
        EPS TEXT,
        RevenuePerShareTTM TEXT,
        ProfitMargin TEXT,
        OperatingMarginTTM TEXT,
        ReturnOnAssetsTTM TEXT,
        ReturnOnEquityTTM TEXT,
        RevenueTTM TEXT,
        GrossProfitTTM TEXT,
        DilutedEPSTTM TEXT,
        QuarterlyEarningsGrowthYOY TEXT,
        QuarterlyRevenueGrowthYOY TEXT,
        AnalystTargetPrice TEXT,
        AnalystRatingStrongBuy TEXT,
        AnalystRatingBuy TEXT,
        AnalystRatingHold TEXT,
        AnalystRatingSell TEXT,
        AnalystRatingStrongSell TEXT,
        TrailingPE TEXT,
        ForwardPE TEXT,overview_arr
        PriceToSalesRatioTTM TEXT,
        PriceToBookRatio TEXT,
        EVToRevenue TEXT,
        EVToEBITDA TEXT,
        Beta TEXT,
        Week52High TEXT,
        Week52Low TEXT,
        MovingAverage50Day TEXT,
        MovingAverage200Day TEXT,
        SharesOutstanding TEXT,
        DividendDate TEXT,
        ExDividendDate TEXT
    )
''')
conn.commit()


In [3]:
overview_arr = []

for ticker in tickers:
    success = False
    attempts = 0
    max_attempts = len(api_keys)

    while not success and attempts < max_attempts:
        current_key = get_current_api_key()
        overview = fetch_company_overview(ticker, current_key)

        # Check if valid data was returned
        if overview != 'no data':
            overview_arr.append(overview)
            success = True
        else:
            print(f"Failed to fetch overview for {ticker} with API key {current_key}")
            rotate_api_key()
            attempts += 1
            disconnect_nordvpn()
            connect_nordvpn()
            # time.sleep(12)  # Respect API rate limit

    if not success:
        print(f"Skipping {ticker} after trying all API keys.")


Fetching data for AAPL using API key QZEDRTVXR9MJO8IW...
Skipped AAPL (no data returned)
Failed to fetch overview for AAPL with API key QZEDRTVXR9MJO8IW
Rotated to new API key: 0EQDLB5ZE6I69MRW
b''
Error disconnecting: b'vpn_disconnect.sh: line 2: nordvpn: command not found\n'


FileNotFoundError: [Errno 2] No such file or directory: 'nordvpn'

In [4]:
import os

# print("PATH:", os.environ["PATH"])


In [14]:
disconnect_nordvpn()

b''
Error disconnecting: b'vpn_disconnect.sh: line 2: nordvpn: command not found\n'


In [2]:
!nordvpn

/bin/sh: line 1: nordvpn: command not found


In [8]:
os.system('which nordvpn')

which: no nordvpn in (/home/shrora/miniconda3/envs/cs571final/bin:/home/shrora/.local/bin:/home/shrora/bin:/app/bin:/app/bin:/app/bin:/usr/bin:/home/shrora/.var/app/com.visualstudio.code/data/node_modules/bin)


256

In [9]:
!python3 helpers.py

b''
Error disconnecting: b'vpn_disconnect.sh: line 2: nordvpn: command not found\n'
None


In [19]:
import sqlite3

conn = sqlite3.connect('company_overview.db')
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(company_overview);")
columns = cursor.fetchall()
for column in columns:
    print(column)
# conn.close()


(0, 'Symbol', 'TEXT', 0, None, 1)
(1, 'AssetType', 'TEXT', 0, None, 0)
(2, 'Name', 'TEXT', 0, None, 0)
(3, 'Description', 'TEXT', 0, None, 0)
(4, 'CIK', 'TEXT', 0, None, 0)
(5, 'Exchange', 'TEXT', 0, None, 0)
(6, 'Currency', 'TEXT', 0, None, 0)
(7, 'Country', 'TEXT', 0, None, 0)
(8, 'Sector', 'TEXT', 0, None, 0)
(9, 'Industry', 'TEXT', 0, None, 0)
(10, 'Address', 'TEXT', 0, None, 0)
(11, 'OfficialSite', 'TEXT', 0, None, 0)
(12, 'FiscalYearEnd', 'TEXT', 0, None, 0)
(13, 'LatestQuarter', 'TEXT', 0, None, 0)
(14, 'MarketCapitalization', 'TEXT', 0, None, 0)
(15, 'EBITDA', 'TEXT', 0, None, 0)
(16, 'PERatio', 'TEXT', 0, None, 0)
(17, 'PEGRatio', 'TEXT', 0, None, 0)
(18, 'BookValue', 'TEXT', 0, None, 0)
(19, 'DividendPerShare', 'TEXT', 0, None, 0)
(20, 'DividendYield', 'TEXT', 0, None, 0)
(21, 'EPS', 'TEXT', 0, None, 0)
(22, 'RevenuePerShareTTM', 'TEXT', 0, None, 0)
(23, 'ProfitMargin', 'TEXT', 0, None, 0)
(24, 'OperatingMarginTTM', 'TEXT', 0, None, 0)
(25, 'ReturnOnAssetsTTM', 'TEXT', 0, Non

In [16]:

cursor.execute('''drop table company_overview''')

In [24]:
cursor.execute('SELECT distinct Symbol FROM company_overview')
rows = list(cursor.fetchall())  # Fetch all results
for row in rows:
    print(row[0])
print(rows)


AAPL
ABBV
ABT
ACN
AMZN
ASML
AVGO
AXP
AZN
BABA
BAC
BRK-A
BRK-B
BX
COST
CRM
CSCO
CVX
DIS
GE
GOOG
GOOGL
HD
HSBC
IBM
JNJ
JPM
KO
LIN
LLY
MA
MCD
META
MRK
MS
MSFT
NFLX
NVDA
NVO
NVS
ORCL
PEP
PG
PLTR
PM
RTX
SAP
SHEL
T
TM
TMO
TMUS
TSLA
TSM
UNH
V
VZ
WFC
WMT
XOM
[('AAPL',), ('ABBV',), ('ABT',), ('ACN',), ('AMZN',), ('ASML',), ('AVGO',), ('AXP',), ('AZN',), ('BABA',), ('BAC',), ('BRK-A',), ('BRK-B',), ('BX',), ('COST',), ('CRM',), ('CSCO',), ('CVX',), ('DIS',), ('GE',), ('GOOG',), ('GOOGL',), ('HD',), ('HSBC',), ('IBM',), ('JNJ',), ('JPM',), ('KO',), ('LIN',), ('LLY',), ('MA',), ('MCD',), ('META',), ('MRK',), ('MS',), ('MSFT',), ('NFLX',), ('NVDA',), ('NVO',), ('NVS',), ('ORCL',), ('PEP',), ('PG',), ('PLTR',), ('PM',), ('RTX',), ('SAP',), ('SHEL',), ('T',), ('TM',), ('TMO',), ('TMUS',), ('TSLA',), ('TSM',), ('UNH',), ('V',), ('VZ',), ('WFC',), ('WMT',), ('XOM',)]
